# Find Game IDs

The objective of this notebook is to take the df with all team stats for each game and append the unique "game_id" to each game using nba_api.

In [1]:
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
import datetime

In [2]:
PATH_TO_TEAM_DATA = "../../data/raw/nba_games_runtime.csv"

In [3]:
df = pd.read_csv(PATH_TO_TEAM_DATA, index_col=0)
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22039,240.0,240.0,44.0,83.0,0.530,13.0,26.0,0.500,12.0,19.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22040,240.0,240.0,46.0,89.0,0.517,11.0,30.0,0.367,25.0,27.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22041,240.0,240.0,31.0,83.0,0.373,8.0,29.0,0.276,21.0,30.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False
22042,240.0,240.0,49.0,92.0,0.533,20.0,42.0,0.476,22.0,28.0,...,66.7,30.8,175.0,146.0,UTA,137,1,2024,2024-02-15,True


In [4]:
# Create a "game_id" column and move to the front
df.insert(0, 'game_id', None)
df

,game_id,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,None,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,None,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,None,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,None,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,None,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22039,None,240.0,240.0,44.0,83.0,0.530,13.0,26.0,0.500,12.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22040,None,240.0,240.0,46.0,89.0,0.517,11.0,30.0,0.367,25.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22041,None,240.0,240.0,31.0,83.0,0.373,8.0,29.0,0.276,21.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False
22042,None,240.0,240.0,49.0,92.0,0.533,20.0,42.0,0.476,22.0,...,66.7,30.8,175.0,146.0,UTA,137,1,2024,2024-02-15,True


In [5]:
# Create a python dict to store team ids
team_ids = dict()

In [6]:
for index, row in df.iterrows(): 
    team_abbrev = row["team"]
    
    if(team_abbrev == "BRK"):
        team_abbrev = "BKN"

    curr_team_id = None

    if(team_abbrev not in team_ids):
        print(team_abbrev)
        curr_team_id = teams.find_team_by_abbreviation(team_abbrev)['id']
        print(f"{team_abbrev}: {curr_team_id}")
        team_ids[team_abbrev] = curr_team_id
    else:
        curr_team_id = team_ids[team_abbrev]


    # Format the game date into MM/DD/YY
    game_date = row["date"]
    formatted_date = datetime.datetime.strptime(game_date, "%Y-%m-%d").strftime("%m/%d/%Y")

    # Now we can fetch the gamelog for that team for that specific date
    gamelog = teamgamelog.TeamGameLog(
        season = 2015,
        season_type_all_star = 'Regular Season',
        team_id = curr_team_id,
        date_from_nullable = formatted_date,
        date_to_nullable = formatted_date
    )

    gamelog_df = gamelog.team_game_log.get_data_frame()
    assert gamelog_df.shape[0] == 1

    game_id = gamelog_df.iloc[0]["Game_ID"]
    row["game_id"] = game_id

DET
DET: 1610612765
ATL
ATL: 1610612737
CLE
CLE: 1610612739
CHI
CHI: 1610612741
NOP
NOP: 1610612740
GSW
GSW: 1610612744
PHI
PHI: 1610612755
BOS
BOS: 1610612738
BKN
BKN: 1610612751
UTA
UTA: 1610612762
DEN
DEN: 1610612743
HOU
HOU: 1610612745
MIN
MIN: 1610612750
LAL
LAL: 1610612747
MEM
MEM: 1610612763
CHO


TypeError: 'NoneType' object is not subscriptable

In [16]:
df[df["team"] == "CHO"]

,game_id,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
18,None,240.0,240.0,33.0,84.0,0.393,6.0,24.0,0.250,22.0,...,50.0,34.1,233.0,111.0,MIA,104,1,2016,2015-10-28,False
40,None,240.0,240.0,36.0,88.0,0.409,12.0,30.0,0.400,10.0,...,23.4,26.3,123.0,105.0,ATL,97,1,2016,2015-10-30,False
81,None,240.0,240.0,32.0,86.0,0.372,14.0,37.0,0.378,14.0,...,37.5,26.6,142.0,110.0,ATL,94,0,2016,2015-11-01,False
105,None,240.0,240.0,47.0,91.0,0.516,14.0,23.0,0.609,22.0,...,80.0,37.4,171.0,140.0,CHI,105,0,2016,2015-11-03,True
140,None,240.0,240.0,44.0,89.0,0.494,6.0,27.0,0.222,14.0,...,50.0,35.0,141.0,118.0,DAL,94,1,2016,2015-11-05,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21913,None,240.0,240.0,44.0,90.0,0.489,16.0,39.0,0.410,13.0,...,33.3,25.1,195.0,128.0,TOR,123,0,2024,2024-02-07,False
21946,None,240.0,240.0,27.0,79.0,0.342,7.0,29.0,0.241,23.0,...,25.0,32.2,201.0,98.0,MIL,120,1,2024,2024-02-09,False
21959,None,240.0,240.0,47.0,82.0,0.573,16.0,31.0,0.516,5.0,...,18.8,37.2,191.0,128.0,MEM,106,0,2024,2024-02-10,True
21983,None,240.0,240.0,44.0,82.0,0.537,10.0,34.0,0.294,13.0,...,30.0,32.1,155.0,120.0,IND,102,0,2024,2024-02-12,True
